# Mask Classification

This Notebook contains a model trainable from scratch a model based on VGG16

In [71]:
import os
import numpy as np 
import tensorflow as tf 
import pandas

SEED = 1234

tf.random.set_seed(SEED)

cwd = os.getcwd();

In [72]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


apply_data_augmentation = True

if apply_data_augmentation: #if data augmentation is enabled, create the generator
     train_data_gen = ImageDataGenerator(rotation_range=10,
                                        zoom_range=0.1,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        validation_split = 0.3,
                                        rescale=1./255)
else: #rescale only the image
     train_data_gen = ImageDataGenerator(rescale = 1./255, validation_split = 0.3)                                       

#rescale only on validation dataset and test dataset
valid_data_gen = ImageDataGenerator(rescale = 1./255)

test_data_gen = ImageDataGenerator(rescale = 1./255)


In [73]:
#width and height of imgaes
img_w = 256
img_h = 256

num_classes = 3

classes = ["NO PERSON", "ALL THE PEOPLE", "SOMEONE"]

bs = 32 #batch size

In [74]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Loading data 
import json 

#setting dirs
dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir = os.path.join(dataset_dir, "training")
validation_dir = training_dir


#reading json file
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)


dataframe = pandas.DataFrame(dic.items())

dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)

dataframe["class"] = dataframe["class"].astype(str)

train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

valid_gen = valid_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)                                                                                                                                                                                     

Found 5614 validated image filenames belonging to 3 classes.
Found 5614 validated image filenames belonging to 3 classes.


In [75]:
#Creating Dataset objects


train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()


valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()     


In [78]:
# Architecture: VGG16 architecture; FC is different in number of neurons.
# Original 4096 neuron per hidden layer

model = tf.keras.Sequential()

#Creating a CNN from scratch
start_f = 8
depth = 5

    

    # Features extraction
for i in range(depth):

    if i == 0:
         input_shape = [img_h, img_w, 3]
    else:
         input_shape=[None]

        # Conv block: Conv2D -> Activation -> Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())                                
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2,2)))

    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [79]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
re_lu_70 (ReLU)              (None, 256, 256, 8)       0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 256, 256, 8)       584       
_________________________________________________________________
re_lu_71 (ReLU)              (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
re_lu_72 (ReLU)              (None, 128, 128, 16)     

In [67]:
# Load the model
load_model = False
if load_model:
    latest = tf.train.latest_checkpoint(os.path.join("C:\\Users\\Giovanni\\Desktop\\Università\\Magistrale\\ANN\\AN2DL-homeworks\\image-classification\\classification_experiments\\CNN_Nov12_08-31-10\\ckpts", ""))
    model.load_weights(latest, by_name=False)

In [80]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [81]:
from datetime import datetime



exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [86]:
model.fit(x=train_dataset,
          epochs=15,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/15
176/176 [==============================] - 186s 1s/step - loss: 0.4789 - accuracy: 0.7868 - val_loss: 0.5251 - val_accuracy: 0.7626
Epoch 2/15
176/176 [==============================] - 179s 1s/step - loss: 0.4767 - accuracy: 0.7870 - val_loss: 0.4616 - val_accuracy: 0.7932
Epoch 3/15
176/176 [==============================] - 185s 1s/step - loss: 0.4716 - accuracy: 0.7932 - val_loss: 0.4277 - val_accuracy: 0.8051
Epoch 4/15
176/176 [==============================] - 171s 969ms/step - loss: 0.4625 - accuracy: 0.7909 - val_loss: 0.5043 - val_accuracy: 0.7786
Epoch 5/15
176/176 [==============================] - 173s 981ms/step - loss: 0.4675 - accuracy: 0.7932 - val_loss: 0.4971 - val_accuracy: 0.7709
Epoch 6/15
176/176 [==============================] - 185s 1s/step - loss: 0.4572 - accuracy: 0.7980 - val_loss: 0.4951 - val_accuracy: 0.7741
Epoch 7/15
176/176 [==============================] - 177s 1s/step - loss: 0.4582 - accuracy: 0.7932 - val_loss: 0.5185 - val_accuracy: 

In [37]:
#check that is all ok

#iterator = iter(valid_dataset)



#from PIL import Image

#for i in range(10):
#    sample, target = next(iterator)
#    sample_ = sample[0, ...]

#    img = Image.fromarray(np.uint8(np.array(sample_)*255.))
#    img = img.resize([img_w,img_h])
#    img_array = np.array(img)
#    img_array = np.expand_dims(img_array, 0) 
#    img_array = tf.cast(img_array, tf.float32) / 255.
#    prediction = model.predict(img_array)
#    img
#    print("Predicted:"+classes[np.argmax(prediction)])
#    print("Original:" +classes[tf.argmax(target[0], axis=0)])

#sample_
#class_names[tf.argmax(target[0], axis=0)]

In [38]:
# Test Dataset 
# Useful if you want to see images with predictions  

#from PIL import Image
#image_filenames = next(os.walk('MaskDataset/test'))[2]

#results = {}
#for image_name in image_filenames:
#   img = Image.open('MaskDataset/test/'+image_name).convert('RGB')
#   img = img.resize((img_w,img_h))
#   img_array = np.array(img)
#   img_array = np.expand_dims(img_array, 0) 
#   img_array = tf.cast(img_array, tf.float32) / 255.
#   prediction = model.predict(img_array)
#   img
#   classes[np.argmax(prediction)]
   


In [87]:
#Creating CSV

import os
from datetime import datetime
from PIL import Image

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


image_filenames = next(os.walk('MaskDataset/test'))[2]

results = {}
for image_name in image_filenames:
   img = Image.open('MaskDataset/test/'+image_name).convert('RGB')
   img = img.resize((img_w,img_h))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 
   img_array = tf.cast(img_array, tf.float32) / 255.
   prediction = model.predict(img_array)
   results[image_name] = np.argmax(prediction)

create_csv(results)            